In [3]:
import numpy as np
import pandas as pd
import scipy
from scipy.spatial import distance
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from scipy.special import rel_entr

In [154]:
Predicted = pd.read_csv("./SpatialDWLS/Outputs/output_Sim8.csv")
Target=pd.read_csv("./Data/Sim8/target.csv")

In [155]:
Predicted

,Unnamed: 0,cell_ID,Endothelial Cell,Thick Ascending Limb,Distal Convoluted Tubule,Proximal Tubule,Connecting Tubule,Principal Cells,Intercalated Cells Type A,Intercalated Cells Type B,Mesangial Cells,Leukocyte
0,1,0,0.000000,0.000000,0.0,0.481801,0.000000,0.301578,0.216621,0.0,0.0,0.000000
1,2,1,0.000000,0.508784,0.0,0.155091,0.000000,0.287620,0.000000,0.0,0.0,0.048505
2,3,2,0.000000,0.000000,0.0,0.669221,0.000000,0.330779,0.000000,0.0,0.0,0.000000
3,4,3,0.123387,0.000000,0.0,0.613679,0.262933,0.000000,0.000000,0.0,0.0,0.000000
4,5,4,0.349909,0.000000,0.0,0.251195,0.000000,0.251076,0.147820,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,995,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
996,997,996,0.000000,0.139500,0.0,0.141753,0.718747,0.000000,0.000000,0.0,0.0,0.000000
997,998,997,0.254979,0.000000,0.0,0.123252,0.000000,0.621768,0.000000,0.0,0.0,0.000000
998,999,998,0.537541,0.000000,0.0,0.462459,0.000000,0.000000,0.000000,0.0,0.0,0.000000


In [156]:
Target

,Unnamed: 0,Proximal Tubule,Thick Ascending Limb,Distal Convoluted Tubule,Connecting Tubule,Principal Cells,Intercalated Cells Type A,Intercalated Cells Type B,Endothelial Cell,Mesangial Cells,Leukocyte
0,0,5.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0
1,1,5.0,4.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
2,2,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
995,995,9.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
996,996,2.0,2.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
997,997,3.0,0.0,0.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0
998,998,9.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0


In [157]:
Predicted.drop(columns="Unnamed: 0", inplace=True)
Predicted.drop(columns="cell_ID", inplace=True)


In [158]:
Target.drop(columns="Unnamed: 0", inplace=True)

In [159]:
for i in range(len(Target)):
    Target.loc[i,:]=Target.loc[i,:]/Target.loc[i,:].sum()

In [160]:
for i in range(len(Predicted)):
    Predicted.loc[i,:]=Predicted.loc[i,:]/Predicted.loc[i,:].sum()
 

In [161]:
columns_p=[]
for i in range(Predicted.shape[1]):
    columns_p.append(Predicted.columns[i])
    

In [162]:
columns_t=[]
for i in range(Target.shape[1]):
    columns_t.append(Target.columns[i])
    

In [163]:
columns_p=sorted(columns_p)
columns_t=sorted(columns_t)

In [164]:
Predicted=Predicted[columns_p]
Target=Target[columns_t]

In [165]:
Target=Target.to_numpy()
Predicted=Predicted.to_numpy()

## Pearson Correlation

In [166]:
num_celltypes=Predicted.shape[1]
d = np.zeros(num_celltypes)
for i in range(0,num_celltypes):
    p=Predicted[:,i]
    t=Target[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]

In [167]:
np.mean(np.nan_to_num(d))

0.6100987204963679

In [168]:
d

array([0.67687559, 0.79467631, 0.72715154, 0.67579752, 0.33706319,
       0.11613477, 0.47030482, 0.69613466, 0.76435158, 0.84249723])

## Jenson Shannon Divergence

In [169]:
jns=distance.jensenshannon(Predicted,Target,axis=1)
jns=jns[~np.isinf(jns).squeeze()]
np.mean(jns)

0.33143927915825167

## F1 Score, Precision, Recall

In [170]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [171]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds
for t in thresholds:
    Predicted_logical=(Predicted>t).astype(int)
    Target_logical=(Target>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [172]:
d

,Precision,Recall,F1-score
0.100,0.729885,0.771898,0.750304
0.200,0.737905,0.707977,0.722631
0.300,0.685714,0.622568,0.652617
0.010,0.734005,0.775300,0.754087
0.001,0.733225,0.775986,0.754000
